This notebook explains initial data management steps.  Functions related to this step can be found in the file_management.py module in utils -> 'utils/file_management.py' or 'from utils import file_management'

Step 1a) This step documents data used in the project.  ScienceBase is used to document source data and retrieval metadata.  When permitted data were downloaded using Python code.  In some cases data needed to be requested and delivered. 

Step 1b) Documents data management steps and concepts to be considered before processing starts.

Step 1c) Create file that documents information about Tiff files.  This file will be used to control processing methods.

Step 1d) Create serlialized versions of HydroSHEDS basin data for quick retrieval of information in processing steps

## Step 1a: Download Source Data
      
#### Standard HydroBASINS from HydroSHEDS: 
    * Store src HydroSHEDS data in repo under 'data/HydroSHEDS'
    * Individual file names need to remain as downloaded
    * Structure under this folder does not matter
    * See more information and download directions here: Coming Soon
    
#### Landscape Variables (var files):
    * Store src landscape variable (var) files in repo under 'data/var'
    * Structure under this folder does not matter
    * See more information and download directions for var files used in this assessment here: Coming Soon
    
#### Source File Information:
    * User created csv file containing processing information about each var file
    * For ease of use name and store file in the repo as: 'data/var/file_processing_info.csv'
    * Required fields for each file include: file_name, variable, src_short, summary_type, label 
    * File name must match corresponsing var file name, shapefiles can be represented by only the .shp file
    * See more information and download directions here: Coming Soon

In [1]:
#Example showing a list of URLS with files for Land Cover data
#Note was having trouble downloading these using Python, server kept cutting off access
import urllib.request as url_r

try:
    target_url = 'https://s3-eu-west-1.amazonaws.com/vito-lcv/2015/ZIPfiles/manifest_cgls_lc_v2_100m_global_2015.txt'
    download_urls = url_r.urlopen(target_url).read()
    download_urls_str = download_urls.decode("utf8")
    list_urls = list(download_urls_str.split('\r\n'))
except Exception as e:
    raise Exception(e)

In [2]:
#Shows first 3 urls
list_urls[0:3]

['https://s3-eu-west-1.amazonaws.com/vito-lcv/2015/ZIPfiles/E000N00_ProbaV_LC100_epoch2015_global_v2.0.1_products_EPSG-4326.zip',
 'https://s3-eu-west-1.amazonaws.com/vito-lcv/2015/ZIPfiles/E000N20_ProbaV_LC100_epoch2015_global_v2.0.1_products_EPSG-4326.zip',
 'https://s3-eu-west-1.amazonaws.com/vito-lcv/2015/ZIPfiles/E000N40_ProbaV_LC100_epoch2015_global_v2.0.1_products_EPSG-4326.zip']

==================================================================================================================
******************************************************************************************************************
==================================================================================================================

## Step 1b: Create intermediate files containing data needed for processing steps

### A) Understand and modify initial data as needed
    * Currently all source data were in the same coordinate system and no tests were built in yet.  User must ensure data are in {'init':'EPSG:4326'}, WGS84.
    * The nodata value must be changed on some grids before the attribution process can be completed.  Not sure what is causing this, as the same nodata value is successfuly used by methods for some grids. Some example grids that needed this step include Pasture2000_5m.tif and Cropland2000_5m.tif.  
    

In [3]:
#Example of how to verify coordinate system

#for raster use rasterio to read data, then use crs method to get coordinate system information
import rasterio
file_name = 'data/var/CroplandPastureArea2000_Geotiff/Pasture2000_5m.tif'
data = rasterio.open(file_name)
coordinate_system = data.crs
print (f'Raster coordinate system {coordinate_system}')

#for shapefile use geopandas, read data in and call crs method, then use crs method to get coordinate system information
import geopandas as gpd
file_name = 'data/HydroSHEDS/basins/hybas_ar_lev12_v1c/hybas_ar_lev12_v1c.shp'
gdf = gpd.read_file(file_name)
gdf_coordinate_system = gdf.crs
print (f'Shapefile coordinate system {gdf_coordinate_system}')

Raster coordinate system EPSG:4326
Shapefile coordinate system {'init': 'epsg:4326'}


In [4]:
#Example of reassigning nodata value for the Pasture2000_5m grid

import rasterio
import numpy as np

file = 'data/var/CroplandPastureArea2000_Geotiff/Pasture2000_5m.tif'
data = rasterio.open(file)
init_data = data.read()
#In this case the initial nodata value is the only negative number, we replace that with the unique value of 999
new_data = np.where(init_data<0, 999, init_data) 

#Use the profile from the orginal data but update the profile to have a nodata value of 999 and when writing
#new file using the new_data where nodata is now represented as 999
with rasterio.Env():
    profile = data.profile
    profile.update(nodata=999)
    
    #save new data to file 'data/var/CroplandPastureArea2000_Geotiff/Pasture2000_5m_update_nodata999.tif'
    with rasterio.open('data/var/CroplandPastureArea2000_Geotiff/Pasture2000_5m_update_nodata999.tif', 'w', **profile) as dst:
        dst.write(new_data)
    


==================================================================================================================
******************************************************************************************************************
==================================================================================================================

## Step 1c) Create a file documenting information about landscape variable files that need processing.  This file provides information so methods know how to handle the files in summarization steps.

    * Creates file: f'{directory}/{short_name}_ file_info.json'  -> example   data/var/tif_var_file_info.json
    
    * Example below creates json file  'data/var/tif_vars_file_info.json'.  The information in this file is used in later steps to inform how tif data from our source landscape variable datasets (stored in data/var directory in the repository) are attributed to basins in HydroSHEDS HYDROBASINS.  
    
    * Currently only gridded information (such as .tif) use this system, but hope to include similar process for vector data 

In [5]:
from utils import file_management as f_mng

directory = 'data/var'
file_type = 'tif'
extension = f'.{file_type}'

short_name = f'{file_type}_vars'
file_list, directory = f_mng.find_files(directory, suffix=extension)
print (file_list)
print (directory)

# convert file information to dictionary and export to json
file_info = f_mng.get_details_all_files(file_list, directory, short_name)

[{'file_path': 'data/var/GWF_N_raster/wfgy_n_mmy_crs.tif', 'file_name': 'wfgy_n_mmy_crs.tif'}, {'file_path': 'data/var/GWF_N_raster/wfgy_n_mmy4326.tif_nodata-999.tif', 'file_name': 'wfgy_n_mmy4326.tif_nodata-999.tif'}, {'file_path': 'data/var/GWF_N_raster/wfgy_n_mmy4326.tif', 'file_name': 'wfgy_n_mmy4326.tif'}, {'file_path': 'data/var/CroplandPastureArea2000_Geotiff/Pasture2000_5m_update_nodata999.tif', 'file_name': 'Pasture2000_5m_update_nodata999.tif'}, {'file_path': 'data/var/CroplandPastureArea2000_Geotiff/Cropland2000_5m.tif', 'file_name': 'Cropland2000_5m.tif'}, {'file_path': 'data/var/CroplandPastureArea2000_Geotiff/Pasture2000_5m.tif', 'file_name': 'Pasture2000_5m.tif'}]
data/var


In [6]:
#Show information about 2 files that were documented in the JSON
file_info[0:2]

[{'file_name': 'wfgy_n_mmy_crs.tif',
  'file_path': 'data/var/GWF_N_raster/wfgy_n_mmy_crs.tif',
  'file_exists': True,
  'meets_criteria': False,
  'in_control_file': True,
  'missing_info': False,
  'bounds': {'xmin': 0.0, 'xmax': 2160.0, 'ymin': 853.0, 'ymax': 0.0},
  'no_data_val': -3.4028234663852886e+38,
  'pixel_size': 1.0,
  'crs': 'EPSG:4326',
  'message': 'wfgy_n_mmy_crs.tif is not in control_file'},
 {'file_name': 'wfgy_n_mmy4326.tif_nodata-999.tif',
  'file_path': 'data/var/GWF_N_raster/wfgy_n_mmy4326.tif_nodata-999.tif',
  'file_exists': True,
  'meets_criteria': False,
  'in_control_file': True,
  'missing_info': False,
  'bounds': {'xmin': -180.0,
   'xmax': 180.00002880000005,
   'ymin': -58.49861145019534,
   'ymax': 83.58473324980466},
  'no_data_val': -999.0,
  'pixel_size': 0.08333334,
  'crs': 'EPSG:4326',
  'message': 'wfgy_n_mmy4326.tif_nodata-999.tif is not in control_file'}]

______________________________________________________________________________________________
#### Below is a count of the number of .tif files ready to be processed and a view of the information for the first 5 files.

Note: File processing information is available in two formats.  The function passes back a list of dictionaries.  Each dictionary stores information about a file.  The same information is also saved to disk as 'data/var/tif_vars_file_info.json', for processing in later sessions.  Here is an example of printing the number of files processed and showing the first 5 records using the returned variable file_info.  

In [7]:
print (f'{len(file_info)} files are documented and ready to process \n')

import pandas as pd
df = pd.DataFrame(file_info[0:5])
df

6 files are documented and ready to process 



,file_name,file_path,file_exists,meets_criteria,in_control_file,missing_info,bounds,no_data_val,pixel_size,crs,message,variable,src_short,summary_type,label,categorical,pixel_inclusion
0,wfgy_n_mmy_crs.tif,data/var/GWF_N_raster/wfgy_n_mmy_crs.tif,True,False,True,False,"{'xmin': 0.0, 'xmax': 2160.0, 'ymin': 853.0, '...",-3.402823e+38,1.000000,EPSG:4326,wfgy_n_mmy_crs.tif is not in control_file,NaN,NaN,NaN,NaN,NaN,NaN
1,wfgy_n_mmy4326.tif_nodata-999.tif,data/var/GWF_N_raster/wfgy_n_mmy4326.tif_nodat...,True,False,True,False,"{'xmin': -180.0, 'xmax': 180.00002880000005, '...",-9.990000e+02,0.083333,EPSG:4326,wfgy_n_mmy4326.tif_nodata-999.tif is not in co...,NaN,NaN,NaN,NaN,NaN,NaN
2,wfgy_n_mmy4326.tif,data/var/GWF_N_raster/wfgy_n_mmy4326.tif,True,False,True,False,"{'xmin': -180.0, 'xmax': 180.00002880000005, '...",-3.402823e+38,0.083333,EPSG:4326,wfgy_n_mmy4326.tif is not in control_file,NaN,NaN,NaN,NaN,NaN,NaN
3,Pasture2000_5m_update_nodata999.tif,data/var/CroplandPastureArea2000_Geotiff/Pastu...,True,True,False,False,"{'xmin': -180.0, 'xmax': 179.99999999997954, '...",9.990000e+02,0.083333,EPSG:4326,NaN,pasture,earthstat_cropland_pasture_2000,count mean nodata,pasture2000,no,all_touching
4,Cropland2000_5m.tif,data/var/CroplandPastureArea2000_Geotiff/Cropl...,True,False,True,False,"{'xmin': -180.0, 'xmax': 179.99999999997954, '...",-3.402823e+38,0.083333,EPSG:4326,Cropland2000_5m.tif is not in control_file,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
len(file_info)

6

In [9]:
#Examples where file exists but the file_processing_information.csv file does not include the files, shows 3 records 
df.loc[(df['in_control_file']==False) & (df['file_exists']==True)]

,file_name,file_path,file_exists,meets_criteria,in_control_file,missing_info,bounds,no_data_val,pixel_size,crs,message,variable,src_short,summary_type,label,categorical,pixel_inclusion
3,Pasture2000_5m_update_nodata999.tif,data/var/CroplandPastureArea2000_Geotiff/Pastu...,True,True,False,False,"{'xmin': -180.0, 'xmax': 179.99999999997954, '...",999.0,0.083333,EPSG:4326,NaN,pasture,earthstat_cropland_pasture_2000,count mean nodata,pasture2000,no,all_touching


==================================================================================================================
******************************************************************************************************************
==================================================================================================================

# Step 1d) Create serlialized versions of HydroSHEDS level 12 basins 

3 variations are created and pickled for future use allowing for quick access to consistent information throughout processing steps
    * geodataframe containing all attributes and all geospatial information (poly)
    * dataframe containing all attributes except geospatial information -> requires less RAM and loads faster when geospatial data are not needed
    * list of hybas_ids 

In [10]:
from utils import file_management as f_mng
f_mng.build_basin_data(level='12', version='v1c', directory = 'data/HydroSHEDS')
#Prints number of records

Missing 8 regional files.
Warning, expected 1034083 ids, you have 46882


In [11]:
#Test read gdf, show first 2 rows
pkl_gdf = 'data/basins_lvl12_gdf.pkl'
gdf = f_mng.read_pkl_gdf(pkl_gdf)
gdf.head(2)

,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,PFAF_ID,ENDO,COAST,ORDER,SORT,geometry
0,8120000010,0,8120000010,8120000010,0.0,0.0,1.8,1.8,811101001000,0,1,0,1,"POLYGON ((-137.96250 58.89167, -137.96471 58.8..."
1,8120000020,0,8120000020,8120000020,0.0,0.0,164.4,164.5,811101002000,0,0,1,2,"POLYGON ((-137.98750 58.90833, -137.98684 58.9..."


In [12]:
#There should be 1034083 rows and 14 columns
gdf.shape

(46882, 14)

In [13]:
#Test read df, show first 3 rows
pkl_df_path = 'data/basins_lvl12_df.pkl'
df=f_mng.read_pkl_df(pkl_df_path)
df.head(2)

,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,PFAF_ID,ENDO,COAST,ORDER,SORT
0,8120000010,0,8120000010,8120000010,0.0,0.0,1.8,1.8,811101001000,0,1,0,1
1,8120000020,0,8120000020,8120000020,0.0,0.0,164.4,164.5,811101002000,0,0,1,2


In [14]:
#There should be 1034083 rows and 13 columns (same as gdf but no geometry)
df.shape

(46882, 13)

In [15]:
#Test read list of HydroBASIN IDs, print first five
pkl_df_path = 'data/basins_lvl12.txt'
hybas_id_list=f_mng.read_pkl_df(pkl_df_path)
hybas_id_list[0:5]

[8120434690, 8120303620, 8120172550, 8120041480, 8120434700]